In [1]:
import duckdb
import pandas as pd
import numpy as np
import json

In [115]:
db = duckdb.connect('data/ufcstats.db')
fight_df = db.execute('''drop view raw.ufcstats_singlefights''').fetchdf()
# fighter_df = db.execute('SELECT * FROM ufcstatsfightertable').fetchdf()

In [104]:
fight_df.columns

Index(['eventname', 'eventdate', 'eventlocation', 'id', 'weightclass',
       'method', 'round', 'time', 'timeformat', 'referee', 'fighter1',
       'fighter1nickname', 'fighter2', 'fighter2nickname', 'fighter1kd',
       'fighter2kd', 'fighter1sigstr', 'fighter2sigstr', 'fighter1sigstrpct',
       'fighter2sigstrpct', 'fighter1totalstr', 'fighter2totalstr',
       'fighter1td', 'fighter2td', 'fighter1tdpct', 'fighter2tdpct',
       'fighter1subatt', 'fighter2subatt', 'fighter1rev', 'fighter2rev',
       'fighter1ctrl', 'fighter2ctrl', 'fighter1head', 'fighter2head',
       'fighter1body', 'fighter2body', 'fighter1leg', 'fighter2leg',
       'fighter1distance', 'fighter2distance', 'fighter1clinch',
       'fighter2clinch', 'fighter1ground', 'fighter2ground', 'fighter1id',
       'fighter2id', 'winningfighter', 'perf', 'sub', 'fight', 'ko', 'judge1',
       'judge1_score', 'judge2', 'judge2_score', 'judge3', 'judge3_score'],
      dtype='object')

In [116]:
db.execute('''
CREATE OR REPLACE TABLE raw.ufcstats_singlefights AS(
    WITH F1 AS (
        SELECT
            eventname,
            eventdate,
            eventlocation,
            id,
            weightclass,
            method,
            round,
            time,
            timeformat,
            referee,
            fighter1id fighterid,
            fighter1 fightername,
            fighter1nickname fighternickname,
            fighter1kd fighterkd,
            fighter1sigstr fightersigstr,
            fighter1sigstrpct fightersigstrpct,
            fighter1totalstr fightertotalstr,
            fighter1td fightertd,
            fighter1tdpct fightertdpct,
            fighter1subatt fightersubatt,
            fighter1rev fighterrev,
            fighter1ctrl fighterctrl,
            fighter1head fighterhead,
            fighter1body fighterbody,
            fighter1leg fighterleg,
            fighter1distance fighterdistance,
            fighter1clinch fighterclinch,
            fighter1ground fighterground,
            fighter2id opponentid,
            winningfighter,
            perf,
            sub,
            fight,
            ko,
            judge1,
            judge1_score,
            judge2,
            judge2_score,
            judge3,
            judge3_score
        FROM
            raw.ufcstatsfighttable)
    SELECT
        eventname,
        eventdate,
        eventlocation,
        id,
        weightclass,
        method,
        round,
        time,
        timeformat,
        referee,
        fighter2id fighterid,
        fighter2 fightername,
        fighter2nickname fighternickname,
        fighter2kd fighterkd,
        fighter2sigstr fightersigstr,
        fighter2sigstrpct fightersigstrpct,
        fighter2totalstr fightertotalstr,
        fighter2td fightertd,
        fighter2tdpct fightertdpct,
        fighter2subatt fightersubatt,
        fighter2rev fighterrev,
        fighter2ctrl fighterctrl,
        fighter2head fighterhead,
        fighter2body fighterbody,
        fighter2leg fighterleg,
        fighter2distance fighterdistance,
        fighter2clinch fighterclinch,
        fighter2ground fighterground,
        fighter1id opponentid,
        winningfighter,
        perf,
        sub,
        fight,
        ko,
        judge1,
        judge1_score,
        judge2,
        judge2_score,
        judge3,
        judge3_score
    FROM
        raw.ufcstatsfighttable
    UNION ALL
    SELECT * FROM F1)''').fetchdf()

,Count
0,14312


In [132]:
db.execute(
    '''
CREATE OR REPLACE TABLE silver.ufcstats_singlefights AS (
    WITH round_cte AS (
        SELECT
            id,
            CASE
                WHEN TIMEFORMAT == 'No Time Limit' THEN NULL
                ELSE [val::int for val in STR_SPLIT(REPLACE(REPLACE(REGEXP_EXTRACT(timeformat, '\((.*)\)'), '(', ''),')',''), '-')]
            END AS rnds                
        FROM
            raw.ufcstats_singlefights
    )

    SELECT
        eventname,
        STRPTIME(eventdate, '%B %d, %Y') AS eventdate,
        STR_SPLIT(eventlocation, ', ')[1] AS eventcity,
        STR_SPLIT(eventlocation, ', ')[2] AS eventstate,
        STR_SPLIT(eventlocation, ', ')[3] AS eventcountry,
        id as fightid,
        REPLACE(weightclass, ' Bout', '') weightclass,

        CASE
            WHEN 
                CONTAINS(weightclass, 'Title') THEN True::BOOLEAN
            ELSE
                False::BOOLEAN
        END AS titlefight,
        method,
        round,
        -- Total rounds in fight
        LIST_COUNT(rcte.rnds) AS totalrounds,
        LIST_SUM(rcte.rnds) * 60 AS totaltime,
        CASE
            WHEN round = 1 THEN str_split(time, ':')[1]::int * 60 + str_split(time, ':')[2]::int
            ELSE LIST_SUM(rcte.rnds[1:round::int - 1]) * 60 + str_split(time, ':')[1]::int * 60 + str_split(time, ':')[2]::int
        END AS time,
        referee,
        fightername,
        STR_SPLIT(fighterid, '/')[5] fighterid,
        STR_SPLIT(opponentid, '/')[5] opponentid,
        fighternickname,
        CASE
            WHEN CONTAINS(fighterkd, '-') THEN NULL
            ELSE fighterkd::int
        END AS fighterkd,
        CASE
            WHEN CONTAINS(fightersigstr, '-') THEN NULL
            ELSE STR_SPLIT(fightersigstr, ' of ')[1]::int 
        END AS fightersigstr,
        CASE
            WHEN CONTAINS(fightersigstr, '-') THEN NULL
            ELSE STR_SPLIT(fightersigstr, ' of ')[2]::int 
        END AS fightersigstratt,
        CASE
            WHEN CONTAINS(fightersigstrpct, '-') OR fightersigstrpct == '' THEN NULL 
            ELSE REGEXP_EXTRACT(fightersigstrpct, '([0-9]+)')::int / 100
        END AS fightersigstrpct,
        CASE
            WHEN CONTAINS(FIGHTERTOTALSTR, '-') THEN NULL
            ELSE STR_SPLIT(FIGHTERTOTALSTR, ' of ')[1]::int
        END AS fightertotalstr,
        CASE
            WHEN CONTAINS(FIGHTERTOTALSTR, '-') THEN NULL
            ELSE STR_SPLIT(FIGHTERTOTALSTR, ' of ')[2]::int
        END AS fightertotalstratt,
        CASE
            WHEN CONTAINS(FIGHTERTD, '-') THEN NULL
            ELSE STR_SPLIT(FIGHTERTD, ' of ')[1]::int
        END AS fightertd,
        CASE
            WHEN CONTAINS(FIGHTERTD, '-') THEN NULL
            ELSE STR_SPLIT(FIGHTERTD, ' of ')[2]::int
        END AS fightertdatt,
        CASE
            WHEN CONTAINS(fightertdpct, '-') OR fightertdpct == '' THEN NULL 
            ELSE REGEXP_EXTRACT(fightertdpct, '([0-9]+)')::int / 100
        END AS fightertdpct,
        CASE
            WHEN CONTAINS(fightersubatt, '-') THEN NULL
            ELSE fightersubatt::int
        END AS fightersubatt,
        CASE
            WHEN CONTAINS(fighterrev, '-') THEN NULL
            ELSE fighterrev::int
        END AS fighterrev,
        CASE
            WHEN CONTAINS(fighterctrl, '-') THEN NULL
            ELSE STR_SPLIT(fighterctrl, ':')[1]::int * 60 + STR_SPLIT(fighterctrl, ':')[2]::int
        END AS fighterctrl,
        CASE
            WHEN CONTAINS(fighterhead, '-') THEN NULL
            ELSE STR_SPLIT(fighterhead, ' of ')[1]::int
        END AS fighterhead,
        CASE
            WHEN CONTAINS(fighterhead, '-') THEN NULL
            ELSE STR_SPLIT(fighterhead, ' of ')[2]::int
        END AS fighterheadatt,
        CASE
            WHEN CONTAINS(fighterbody, '-') THEN NULL
            ELSE STR_SPLIT(fighterbody, ' of ')[1]::int
        END AS fighterbody,
        CASE
            WHEN CONTAINS(fighterbody, '-') THEN NULL
            ELSE STR_SPLIT(fighterbody, ' of ')[2]::int
        END AS fighterbodyatt,
        CASE
            WHEN CONTAINS(fighterleg, '-') THEN NULL
            ELSE STR_SPLIT(fighterleg, ' of ')[1]::int
        END AS fighterleg,
        CASE
            WHEN CONTAINS(fighterleg, '-') THEN NULL
            ELSE STR_SPLIT(fighterleg, ' of ')[2]::int
        END AS fighterlegatt,
        CASE
            WHEN CONTAINS(fighterdistance, '-') THEN NULL
            ELSE STR_SPLIT(fighterdistance, ' of ')[1]::int
        END AS fighterdistance,
        CASE
            WHEN CONTAINS(fighterdistance, '-') THEN NULL
            ELSE STR_SPLIT(fighterdistance, ' of ')[2]::int
        END AS fighterdistanceatt,
        CASE
            WHEN CONTAINS(fighterclinch, '-') THEN NULL
            ELSE STR_SPLIT(fighterclinch, ' of ')[1]::int
        END AS fighterclinch,
        CASE
            WHEN CONTAINS(fighterclinch, '-') THEN NULL
            ELSE STR_SPLIT(fighterclinch, ' of ')[2]::int
        END AS fighterclinchatt,
        CASE
            WHEN CONTAINS(fighterground, '-') THEN NULL
            ELSE STR_SPLIT(fighterground, ' of ')[1]::int
        END AS fighterground,
        CASE
            WHEN CONTAINS(fighterground, '-') THEN NULL
            ELSE STR_SPLIT(fighterground, ' of ')[2]::int
        END AS fightergroundatt,
        CASE
            WHEN fighterid = winningfighter THEN True::BOOLEAN
            ELSE False::BOOLEAN
        END AS winner,
        CASE
            WHEN fighterid = winningfighter AND perf != '---' THEN True::BOOLEAN
            ELSE False::BOOLEAN
        END AS performancebonus,
        CASE
            WHEN fighterid = winningfighter AND sub != '---' THEN True::BOOLEAN
            ELSE False::BOOLEAN
        END AS submissionbonus,
        CASE
            WHEN fighterid = winningfighter AND ko != '---' THEN True::BOOLEAN
            ELSE False::BOOLEAN
        END AS knockoutbonus,
        CASE
            WHEN fight != '---' THEN True::BOOLEAN
            ELSE False::BOOLEAN
        END AS fightofthenight,
        judge1,
        CASE
            WHEN judge1_score = '---' THEN NULL
            ELSE STR_SPLIT(judge1_score, ' - ')[1]::int
        END AS judge1fighter1score,
        CASE
            WHEN judge1_score = '---' THEN NULL
            ELSE STR_SPLIT(judge1_score, ' - ')[2]::int
        END AS judge1fighter2score,
        judge2,
        CASE
            WHEN judge2_score = '---' THEN NULL
            ELSE STR_SPLIT(judge2_score, ' - ')[1]::int
        END AS judge2fighter1score, 
        CASE
            WHEN judge2_score = '---' THEN NULL
            ELSE STR_SPLIT(judge2_score, ' - ')[2]::int
        END AS judge2fighter2score,
        judge3,
        CASE
            WHEN judge3_score = '---' THEN NULL
            ELSE STR_SPLIT(judge3_score, ' - ')[1]::int
        END AS judge3fighter1score,
        CASE
            WHEN judge3_score = '---' THEN NULL
            ELSE STR_SPLIT(judge3_score, ' - ')[2]::int
        END AS judge3fighter2score
    FROM
        raw.ufcstats_singlefights main
    INNER JOIN round_cte rcte USING (id)
)
'''
).fetchdf()

,Count
0,28624


In [133]:
fight_df = db.execute('''select * from silver.ufcstats_singlefights''').fetchdf()

In [31]:
db.close()